In [ ]:
import pandas as pd
import numpy as np

# Clean cust_info.csv file
1. fix the datatype
2. Check for missing values
3. Check for duplicated values
4. remove space
5. convert 'F' to 'Female', 'M' to 'Male' and 'N/A' for others
convert 'S' to 'Single', 'M' to 'Married'


In [ ]:
customer_df = pd.read_csv('cust_info.csv')
customer_df.shape

(18494, 7)

check for null value

In [ ]:
customer_df.isnull().sum()

In [ ]:
customer_df=customer_df[~customer_df.cst_id.isnull()]

In [ ]:
customer_df.cst_id = customer_df.cst_id.astype(int)

In [ ]:
customer_df.info()

check for duplicate

In [ ]:
duplicated_ids = customer_df.loc[customer_df.cst_id.duplicated(), 'cst_id'].values.tolist()
duplicated_ids

In [ ]:
customer_df[customer_df.cst_id.isin(duplicated_ids) ].head()

In [ ]:
customer_df['rank'] = customer_df.groupby('cst_id')["cst_create_date"].rank(method='dense', ascending=False)

In [ ]:
customer_df = customer_df.loc[customer_df['rank']==1]
customer_df.shape

In [ ]:
customer_df[customer_df.cst_id.isin(duplicated_ids) ]
customer_df.drop('rank', axis=1, inplace=True)

Remove white space in column name or cell value if there are

In [ ]:
customer_df.columns = customer_df.columns.str.strip()

In [ ]:
customer_df = customer_df.applymap(lambda x : x.strip() if isinstance(x, str) else x)

In [ ]:
customer_df.cst_gndr = customer_df.cst_gndr.replace({'F':'Female', 'M': 'Male', np.nan: 'N/A'  })

In [ ]:
customer_df.cst_gndr.unique()

In [ ]:
customer_df.cst_marital_status = customer_df.cst_marital_status.replace({'M': 'Married', 'S': 'Single'})

# Clean Product file

1. Check for missing values
2. Check for duplicated values
3. remove space
4. split the prd_key to cat_id and prd_sub_key
5. replace hyphen with underscore to match cat in erp_px_cat table
6. Replace null cost with 0
7. fill the na value in end_dt column with row below in start_dt column

In [ ]:
product_df = pd.read_csv("prd_info.csv")
product_df.shape

(397, 7)

In [ ]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   prd_id        397 non-null    int64  
 1   prd_key       397 non-null    object 
 2   prd_nm        397 non-null    object 
 3   prd_cost      395 non-null    float64
 4   prd_line      380 non-null    object 
 5   prd_start_dt  397 non-null    object 
 6   prd_end_dt    200 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 21.8+ KB


In [ ]:
# product_df.prd_start_dt = pd.to_datetime(product_df.prd_start_dt)

In [ ]:
# np.where(product_df.prd_cost.isnull(), 0, product_df.prd_cost)
product_df.prd_cost = product_df.prd_cost.fillna(0)

In [ ]:
product_df.prd_line = product_df.prd_line.replace({
    'M': 'Mountain',
    'R':'Road',
    'S':'Other sales',
    'T': 'Touring',
    np.nan : 'N/A'
})

In [ ]:
product_df.prd_end_dt = product_df.prd_end_dt\
                        .fillna(product_df.prd_start_dt.shift(-1))\
                        .fillna(product_df.prd_start_dt.shift(1) ) #if the last row is empty, use the previous row

In [ ]:
product_df.isnull().sum()

,0
prd_id,0
prd_key,0
prd_nm,0
prd_cost,0
prd_line,0
prd_start_dt,0
prd_end_dt,0


In [ ]:
product_df[product_df.prd_id.duplicated()]

,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt


In [ ]:
product_df = product_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipython-input-13-3901201199.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  product_df = product_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
product_df[['prd_cat_id', 'prd_sub_key']]=product_df.prd_key.str.extract(f'(\w+-\w+)-(.*)')
product_df.prd_cat_id = product_df.prd_cat_id.replace("-","_")

# Clean Sales table

1. troubleshotting when sales price is null, when sales number is negative or when sales != quantity * price

In [ ]:
sales_df = pd.read_csv("sales_details.csv")
sales_df.shape

(60398, 9)

In [ ]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sls_ord_num   60398 non-null  object 
 1   sls_prd_key   60398 non-null  object 
 2   sls_cust_id   60398 non-null  int64  
 3   sls_order_dt  60398 non-null  int64  
 4   sls_ship_dt   60398 non-null  int64  
 5   sls_due_dt    60398 non-null  int64  
 6   sls_sales     60390 non-null  float64
 7   sls_quantity  60398 non-null  int64  
 8   sls_price     60391 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 4.1+ MB


In [ ]:
sales_df = sales_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipython-input-17-473462811.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sales_df = sales_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
sales_df.columns = sales_df.columns.str.strip()

In [ ]:
#ensure price value is positive, quantity is positive
sales_df['sls_price'] = abs(sales_df.sls_price)

# if sales price is zero or null, calculate using sales and quantity
sales_df['sls_price'] = np.where(sales_df.sls_price.isnull(), sales_df.sls_sales/sales_df.sls_quantity, sales_df.sls_price)
# if sales is negative, recalculate from quantity and price
sales_df['sls_sales'] = np.where((sales_df['sls_sales'] <= 0) | (sales_df['sls_sales'].isnull()), sales_df.sls_quantity*sales_df.sls_price, sales_df.sls_sales)


In [ ]:
sales_df[sales_df.sls_price<=0]
sales_df[sales_df.sls_quantity<=0]
sales_df[sales_df.sls_sales<=0]
sales_df[sales_df.sls_sales.isnull()]

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price


In [ ]:
sales_df.isnull().sum()

,0
sls_ord_num,0
sls_prd_key,0
sls_cust_id,0
sls_order_dt,0
sls_ship_dt,0
sls_due_dt,0
sls_sales,0
sls_quantity,0
sls_price,0


convert to data type

In [ ]:
sales_df['sls_ship_dt']=pd.to_datetime(sales_df.sls_ship_dt, format='%Y%m%d')
sales_df['sls_due_dt'] = pd.to_datetime(sales_df.sls_due_dt, format='%Y%m%d')
# sales_df['sls_order_dt'] = pd.to_datetime(sales_df.sls_order_dt, format='%Y%m%d')

# Clean erp cust table

In [ ]:
cust_df = pd.read_csv("CUST_AZ12.csv")
cust_df.shape

(18484, 3)

In [ ]:
cust_df.CID=cust_df.CID.replace(r'^NAS','', regex=True)

In [ ]:
cust_df = cust_df.applymap(lambda x : x.strip() if isinstance(x, str) else x)

/tmp/ipython-input-26-890709521.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cust_df = cust_df.applymap(lambda x : x.strip() if isinstance(x, str) else x)


In [ ]:
cust_df.GEN = cust_df.GEN.replace({'F':'Female', 'M':'Male', np.nan:'N/A', '':'N/A'})

In [ ]:
cust_df.GEN.unique()

array(['Male', 'Female', 'N/A'], dtype=object)

In [ ]:
cust_df.isnull().sum()

,0
CID,0
BDATE,0
GEN,0


# Clean erp loc file

In [ ]:
loc_df = pd.read_csv('LOC_A101.csv')
loc_df.shape

(18484, 2)

In [ ]:
loc_df=loc_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipython-input-92-3157779679.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loc_df=loc_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
loc_df.CID = loc_df.CID.str.replace('-','')


In [ ]:
loc_df.CNTRY = loc_df.CNTRY.replace({
np.nan: 'N/A',
'':'N/A',
'DE':'Germany',
'US':'United States',
'USA':'United States'})

In [ ]:
loc_df.CNTRY.unique()

array(['Australia', 'United States', 'Canada', 'Germany',
       'United Kingdom', 'France', 'N/A'], dtype=object)

In [ ]:
loc_df[loc_df.CNTRY.isnull()]

,CID,CNTRY


In [ ]:
loc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CID     18484 non-null  object
 1   CNTRY   18484 non-null  object
dtypes: object(2)
memory usage: 288.9+ KB


In [ ]:
loc_df.isnull().sum()

,0
CID,0
CNTRY,0


# Clean erp Category table

In [ ]:
cat_df = pd.read_csv("PX_CAT_G1V2.csv")
cat_df.shape

(37, 4)

In [ ]:
cat_df.head()

,ID,CAT,SUBCAT,MAINTENANCE
0,AC_BR,Accessories,Bike Racks,Yes
1,AC_BS,Accessories,Bike Stands,No
2,AC_BC,Accessories,Bottles and Cages,No
3,AC_CL,Accessories,Cleaners,Yes
4,AC_FE,Accessories,Fenders,No


In [ ]:
cat_df.isnull().sum()

,0
ID,0
CAT,0
SUBCAT,0
MAINTENANCE,0
